#  Radio Frequency Interference mitigation using deep convolutional neural networks

### This example demonstrates how tf_unet is trained on the 'Bleien Galactic Survey data'. 

To create the training data the SEEK package (https://github.com/cosmo-ethz/seek) has to be installed

In [ ]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import glob
plt.rcParams['image.cmap'] = 'gist_earth'

## preparing training data
only one day...

In [ ]:
!wget -q -r -nH -np --cut-dirs=2 https://people.phys.ethz.ch/~ipa/cosmo/bgs_example_data/

In [ ]:
!mkdir -p bgs_example_data/seek_cache

In [ ]:
!seek --file-prefix='./bgs_example_data' --post-processing-prefix='bgs_example_data/seek_cache' --chi-1=20 --overwrite=True seek.config.process_survey_fft

## setting up the unet

In [ ]:
from scripts.radio_util import DataProvider
from tf_unet import unet


In [ ]:
files = glob.glob('bgs_example_data/seek_cache/*')

In [ ]:
data_provider = DataProvider(600, files)

net = unet.Unet(channels=data_provider.channels, 
                n_class=data_provider.n_class, 
                layers=3, 
                features_root=64,
                cost_kwargs=dict(regularizer=0.001),
                )


## training the network
only one epoch. For good results many more are neccessary

In [ ]:

trainer = unet.Trainer(net, optimizer="momentum", opt_kwargs=dict(momentum=0.2))
path = trainer.train(data_provider, "./unet_trained_bgs_example_data", 
                     training_iters=32, 
                     epochs=1, 
                     dropout=0.5, 
                     display_step=2)



## running the prediction on the trained unet

In [ ]:
data_provider = DataProvider(10000, files)
x_test, y_test = data_provider(1)
prediction = net.predict(path, x_test)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(12,4))
ax[0].imshow(x_test[0,...,0], aspect="auto")
ax[1].imshow(y_test[0,...,1], aspect="auto")
ax[2].imshow(prediction[0,...,1], aspect="auto")